# Validate Ionospheric Delay calibration

In [ ]:
import os
import h5py
%matplotlib inline

In [ ]:
## required to provide path to notebook utils
import sys

sys.path.insert(0, "../")

In [ ]:
from notebook_utils import identify_max_min_baselineid, plot_phase_vs_time, plot_time_vs_freq_for_phase, plot_time_vs_freq_for_phase_multiple_baselines

## Data Generation

We had simulated data using Oskar. The simulation scripts are present in `scripts/ska_low_sim`. (refer [confluence page](https://confluence.skatelescope.org/display/SE/DHR-311%3A+Script+to+simulate+SKA-LOW+visibilities))


For simulation following configuration is used. (further refered as `ionospheric_delay_cal.yaml`)

``` yaml
scenario: "ionodelaycal"          # Scenario name (used for output folder prefix)

# ===============================
# Global simulation parameters
# ===============================

n_stations: 40                                         # Number of stations
tel_model: "./telescope-models/SKA-Low_AA2_40S_rigid-rotation_model.tm" # Telescope model directory

simulation_start_frequency_hz: 123.0e6                  # Start frequency (Hz)
simulation_end_frequency_hz: 153.0e6                    # End frequency (Hz)
correlated_channel_bandwidth_hz: 21.70138888888889e3    # Channel width (Hz)

observing_time_mins: 10                              # Observation duration (minutes)
sampling_time_sec: 3.3973862400000003                   # Dump/integration time (seconds)

fields:
  EoR2:
    Cal1:
      ra_deg: 197.914612
      dec_deg: -22.277973
      scan_id_start: 300
      transit_time: "2000-01-03 22:33:30.000"

# ==================================
# Options for generate_gaintable.py
# ==================================

generate_gaintable:
  output_gaintable: &gen_gaintable "./gaintables/custom_gaintable.h5"

  station_offset: true              # Apply per-station amplitude/phase offsets
  time_variant: true                # Apply time-dependent effects

  rfi: false                        # Inject RFI band
  rfi_start_freq_hz: 154.25347222228538e6        # Hz
  rfi_end_freq_hz: 159.8090277778474e6           # Hz

  plot: true                        # Generate diagnostic plots
  plot_output_dir: "./gaintables/generation_plots/"

# ===============================
# Options for run_sim.py
# ===============================

run_sim:
  oskar_sif: "./OSKAR-2.11.1-Python3.sif" # Path to OSKAR Singularity image

  # GLEAM sky model. Optional. Comment to disable.
  gleam_file: "./sky-models/GLEAM_EGC.fits" # GLEAM catalogue FITS file
  field_radius_deg: 10.0            # Radius of field of view (degrees)

  # Corruptions to be applied. All are optional. Comment to disable.
  # gaintable: *gen_gaintable           # Gaintable containing bandpass corruptions
  #cable_delay: "./cable_delays/cable_length_error_40s.txt" # Cable delay error file
  tec_screen: "./tec/calibrator_iono_tec.fits" # Ionospheric TEC screen FITS

  # Imaging parameters using wsclean. Optional. Comment to disable.
  create_dirty_image: true          # Whether to run wsclean imaging
  image_size: 1024                  # Image size (pixels)
  pixel_size: "2arcsec"             # Pixel size (angular units)

  # Extra parameters to pass directly to run_oskar.py
  run_oskar_extra_params: "--use-gpus --double-precision"
```

Follow steps mentioned in confluence page for data and enviornment setup. Run the following command to simulate visibilities. 
`python run_sim.py ionospheric_delay_cal.yaml`


## Pipeline Setup

In [ ]:
cache = "../../cache"
artifacts_prefix_path = "./ionospheric_cal_artefacts"
os.makedirs(artifacts_prefix_path, exist_ok=True)

Running the pipeline with simulated data with ionoshperic effects.

Please replace data paths with correct ones in use.

If instrumental pipeline fails to run, please try to run just the pipeline in a separate enviroment having only pipeline dependencies. You can copy paste the command from here.

In [ ]:
input_data = "/home/ska/Work/data-simulation/simulations/simulated_data/ionospheric_delay/visibility.scan-300.ms"

In [ ]:

## Running the pipeline
!ska-sdp-instrumental-calibration run \
    --input $input_data \
    --stages "load_data,predict_vis,ionospheric_delay" \
    --set parameters.load_data.cache_directory $cache \
    --set parameters.predict_vis.beam_type "everybeam" \
    --set parameters.predict_vis.normalise_at_beam_centre true \
    --set parameters.predict_vis.eb_coeffs "/home/ska/Work/data/INST/sim/coeffs"  \
    --set parameters.predict_vis.lsm_csv_path "/home/ska/Work/data-simulation/simulations/simulated_data/ionospheric_delay/sky_model.csv" \
    --set parameters.ionospheric_delay.niter 100 \
    --set parameters.ionospheric_delay.plot_table true \
    --set parameters.ionospheric_delay.export_gaintable true \
    --dask-scheduler "tcp://10.131.131.55:8786" \
    --output $artifacts_prefix_path \
    --no-unique-output-subdir

## Apply gaintable to MS using target bpp

In [ ]:
gaintable = f"{artifacts_prefix_path}/gaintables/ionospheric_delay.gaintable.h5parm"

In [ ]:
data_output_dir = f"{artifacts_prefix_path}/data"
os.makedirs(data_output_dir, exist_ok=True)

In [ ]:
target_bpp_config = f"""
steps:
- applycal:
    parmdb: {gaintable}
- averager:
    freqstep: 1
    timestep: 1
"""

In [ ]:
from tempfile import NamedTemporaryFile


config_file = NamedTemporaryFile(mode="w", encoding="utf-8", delete=False)
config_file.write(target_bpp_config)
filename = config_file.name
config_file.close()

!ska-sdp-batch-preprocess \
    --config $filename \
    --output-dir $data_output_dir \
    $input_data

## Beam Correction

We require to perform beam correction so that point source becomes more visible in the dirty images.

In [ ]:
beam_corrected_input_ms = f"{data_output_dir}/beam_corrected_input.ms"
beam_corrected_applied_ms = f"{data_output_dir}/beam_corrected_applied.ms"

applied_ms = f"{data_output_dir}/visibility.scan-300.ms"

Beam correction using DP3.

In [ ]:
!DP3 msin=$input_data steps=[applybeam] msout=$beam_corrected_input_ms
!DP3 msin=$applied_ms steps=[applybeam] msout=$beam_corrected_applied_ms

## Generate Image

Generate image using wsclean from visibilities before and after ionospheric calibration.

In [ ]:
image_output_dir = f"{artifacts_prefix_path}/images"
os.makedirs(image_output_dir, exist_ok=True)

### Create Original fits image

In [ ]:
original_fits = f"{image_output_dir}/original_fits.ms"
!wsclean -size 512 512 -scale "2arcsec" -niter 0 -name $original_fits $beam_corrected_input_ms

### Create corrected fits image

In [ ]:
corrected_fits = f"{image_output_dir}/corrected"
!wsclean -size 512 512 -scale "2arcsec" -niter 0 -name $corrected_fits $beam_corrected_applied_ms

### Remarks

In ionospheric corrected image, the sources should be shifted a bit - the shift magnitude depends on the TEC screen fits file used in input data simulation.

## Phase vs Time plots

Plotting phase vs time for a given baseline and channel to see if phase looks less scattered in corrected data.

In [ ]:
from ska_sdp_instrumental_calibration.data_managers.visibility import (
    load_ms_as_dataset_with_time_chunks,
)
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
input_vis = load_ms_as_dataset_with_time_chunks(beam_corrected_input_ms, 30).load()
corrected_vis = load_ms_as_dataset_with_time_chunks(beam_corrected_applied_ms, 30).load()

###  Selected baseline and channel

In [ ]:
baseline = 10
channel = 100

plot_phase_vs_time(input_vis,corrected_vis,channel,baseline,artifacts_prefix_path)

### Shortest and longest lengths baselines with central frequency

In [ ]:
long_bl,short_bl = identify_max_min_baselineid(input_vis.uvw)
central_freq = input_vis.frequency.size // 2

In [ ]:
# for shortest baseline
plot_phase_vs_time(input_vis,corrected_vis,central_freq,short_bl,artifacts_prefix_path)

In [ ]:
# for longest baseline
plot_phase_vs_time(input_vis,corrected_vis,central_freq,long_bl,artifacts_prefix_path)

## Time vs freq plot for phase waterfall plots

In [ ]:
plot_time_vs_freq_for_phase(input_vis, corrected_vis, short_bl, artifacts_prefix_path)

In [ ]:
plot_time_vs_freq_for_phase(input_vis, corrected_vis, long_bl, artifacts_prefix_path)

In [ ]:
plot_time_vs_freq_for_phase_multiple_baselines(input_vis, 100, 150, "Original", artifacts_prefix_path)

In [ ]:
plot_time_vs_freq_for_phase_multiple_baselines(corrected_vis, 100, 150, "Corrected", artifacts_prefix_path)